<a href="https://colab.research.google.com/github/ricardofelippe/Mapa/blob/main/Explorando_as_despesas_do_governo_federal_via_API_%5BEscola_de_Dados%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PNCP ATAS

## API x Bases de Dados
---

Quem não tem fluência em programação, pode pasar por uma certa curva de aprendizagem para começar a usar APIs (*Application Programming Interface*). Em alguns casos, porém, você pode ganhar muito tempo se optar por consumir os dados via API, mesmo que demore mais para criar seu código pela primeira vez.

Confira abaixo uma comparação sobre as duas formas de acessar os dados. Mas atenção: essa comparação não é geral, ela se aplica especificamente ao Portal da Transparência do Governo Federal e à maneira como os dados estão disponibilizados hoje.

| Situação                       |   API           | Download de CSV  |
| :-------------                 | :-------------  | :------------- |
| Preciso de uma série histórica grande| +               | +++++          |
| Não faço pauta desse tema com frequência | ++               | +++++|
| Quero monitorar um assunto/cnpj/órgão etc | +++++               | +          |
| Quero conectar em um site ou visualização | ++++               | ++      |
| *Que outras situações ocorrem a vocês?* |               |      |




### Onde encontrar?

Para conhecer os dados disponíveis, sistema de origem, órgão responsável, periodicidade e data de atualização do Portal (download e API):

http://transparencia.gov.br/origem-dos-dados

- **Acesso via API**

http://www.transparencia.gov.br/swagger-ui.html

O Swagger é uma feramenta que documenta e permite explorar o conteúdo. Iremos falar mais sobre ela abaixo.

- **Bases para download**

http://transparencia.gov.br/download-de-dados

https://brasil.io/dataset/gastos-diretos/gastos (coletado e tratado por Álvaro Justen (Turicas) - tenha atenção para as datas de atualização no site)


# Acessando a API

---



Agora que já compreendemos o básico sobre como as despesas do governo federal são registradas, vamos colocar a mão na massa rodando alguns códigos em Python.

Iremos começar importando os pacotes que iremos utilizar.

Clique no botão de "Play" abaixo para executar o código. Se nenhum erro aparecer, então, deu tudo certo :)

In [2]:
import pandas as pd
import requests
import json
import numpy as np
from IPython.display import Image


In [3]:
# Troque os caracteres abaixo por uma chave válida, criada com seu email, usando o endereço abaixo
# http://www.portaldatransparencia.gov.br/api-de-dados/cadastrar-email
minha_chave = '3fd8bbb6a046fd05f768fb0b7fbefbb3'

# para conseguir acessar a API, precisamos incluir no cabeçalho da requisição a nossa chave, que definimos anteriormente. Não é necessário alterar essa variável.
chave = {'chave-api-dados':minha_chave}

## Fonte de Dados   : https://pncp.gov.br/api/consulta/swagger-ui/index.html#/                                                                                                                                   
           

In [ ]:
# Esta URL foi gerada a partir da consulta por órgão no swagger
url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701&pagina=429'
#url = f'https://pncp.gov.br/api/consulta/v1/atas?anoAta=20240807&dataInicial=20240828&dataFinal=20250801&pagina=429'

    # aqui, fazemos a consulta e obtemos o resultado para cada ano em JSON
r_orgao = requests.get(url, verify=True,headers=chave).json()

dados=r_orgao['data']
#r_orgao=dados
totalPaginas=r_orgao['totalPaginas']

"""    # e aqui juntamos os resultados de cada ano em uma série histórica do órgão

# Verifica se a chave 'data' existe
dados = r_orgao.get('data')"""

# Se a chave existir, continue; caso contrário, exiba uma mensagem
if dados:
    r_orgao = dados
else:
    print("A chave 'data' não foi encontrada em r_orgao.")

if totalPaginas>0:
    print(totalPaginas)


In [ ]:
r_orgao

## Criando o DATAFRAME

In [ ]:
dataPublicacaoPncp=list(map(lambda x:x["dataPublicacaoPncp"],r_orgao))
numeroControlePNCPAta=list(map(lambda x:x["numeroControlePNCPAta"],r_orgao))
numeroControlePNCPCompra=list(map(lambda x:x["numeroControlePNCPCompra"],r_orgao))
numeroAtaRegistroPreco=list(map(lambda x:x["numeroAtaRegistroPreco"],r_orgao))
anoAta=list(map(lambda x:x["anoAta"],r_orgao))
#cancelado=list(map(lambda x:x["cancelado"],r_orgao))
vigenciaInicio=list(map(lambda x:x["vigenciaInicio"],r_orgao))
vigenciaFim=list(map(lambda x:x["vigenciaFim"],r_orgao))
usuario=list(map(lambda x:x["usuario"],r_orgao))
objetoContratacao=list(map(lambda x:x["objetoContratacao"],r_orgao))
nomeOrgao=list(map(lambda x:x["nomeOrgao"],r_orgao))
codigoUnidadeOrgao=list(map(lambda x:x["codigoUnidadeOrgao"],r_orgao))
nomeUnidadeOrgao=list(map(lambda x:x["nomeUnidadeOrgao"],r_orgao))

data = pd.DataFrame([dataPublicacaoPncp,numeroControlePNCPAta,numeroControlePNCPCompra,numeroAtaRegistroPreco,
                     anoAta,vigenciaInicio,vigenciaFim,usuario,objetoContratacao,
                      codigoUnidadeOrgao,nomeOrgao,nomeUnidadeOrgao]) # Each list would be added as a row
data = data.transpose() 
data.columns = ['dataPublicacaoPncp','numeroControlePNCPAta','numeroControlePNCPCompra','numeroAtaRegistroPreco',
                'anoAta','vigenciaInicio','vigenciaFim','usuario','objetoContratacao','codigoUnidadeOrgao',
                'nomeOrgao','nomeUnidadeOrgao']

""" data=data.query("usuario == 'Contratos.gov.br'")
data=data.reset_index(drop=True)
 """
 #data

# Organizando a coluna dataPublicacaoPncp em ordem decrescente
data.sort_values(['dataPublicacaoPncp'], ascending=[False])


# Proximos Passos

1- filtrar usuário compras.gov.br
2- Anexar documentos da ata 

In [ ]:
Image("explicacaoNumeracaoPNCP.png")


In [ ]:
# Extraindo subconjunto de colunas
# Index(['numeroControlePNCPAta', 'anoAta', 'cancelado', 'vigenciaInicio',
    #    'vigenciaFim', 'usuario', 'objetoContratacao', 'codigoUnidadeOrgao',
    #    'nomeUnidadeOrgao'],
    #   dtype='object')
#url = f'https://pncp.gov.br/api/pncp/v1/orgaos/00394452000103/compras/2024/011953/atas/000003/arquivos'

data=data[data.objetoContratacao.str.contains("LIMPEZA", case=False)].reset_index(drop=True)
cols = [0,1] # numeroControlePNCPAta, anoAta
df = data[data.columns[cols]]
df

In [ ]:
# URL BASE= https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra/atas/{sequencialAta}/arquivos
# 01612781000138-1-000020/2022-000001
cnpj=df.numeroControlePNCPAta.str[:14]
sequencialCompra=df.numeroControlePNCPAta.str[17:23]
anoCompra=df.numeroControlePNCPAta.str[24:28]
sequencialAta=df.numeroControlePNCPAta.str[29:35]
DocAta=pd.DataFrame([cnpj,anoCompra, sequencialCompra,sequencialAta])
DocAta = DocAta.transpose() 
DocAta.columns = ['cnpj','anoCompra','sequencialCompra','sequencialAta']
DocAta






In [ ]:
DocAta

In [ ]:
i=2
DocAta.loc[i, 'cnpj']
numLinhas=DocAta.shape[0]



In [ ]:
data.shape

In [ ]:
numLinhas=DocAta.shape[0]
urlAtas=[]
#urlAtasHistoricas=[]
for i in range(numLinhas):
    cnpj=DocAta.loc[i, 'cnpj']
    anoCompra=DocAta.loc[i, 'anoCompra']
    sequencialCompra=DocAta.loc[i, 'sequencialCompra']
    sequencialAta=DocAta.loc[i, 'sequencialAta']

    url1 = f'https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/arquivos'
    #url_historico = f'https://pncp.gov.br/api/pncp//v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/historico'
   #url = f'https://pncp.gov.br/api/pncp/v1/orgaos/00394452000103/compras/2024/011953/atas/000003/arquivos' 
    #https://pncp.gov.br/api/pncp//v1/orgaos/00394452000103/compras/2024/011953/atas/000003/historico
    
    # esta URL foi gerada a partir da consulta por órgão no swagger
    #temp=url1[0]['url']
    response = requests.get(url1, verify=True, headers=chave)
    #response2 = requests.get(url_historico, verify=True, headers=chave)
    
    #response = requests.get(url1)

    #print(response.text)

    # Verifica se o tipo de conteúdo é JSON
    
    if 'application/json' in response.headers.get('Content-Type', ''):
        if response.content:
            #data = response.json()  # Tenta parsear como JSON
            temp = response.json()
            temp=temp[0]['url']
            #print(temp)
        else:
            temp=""        
    else:
        temp=""
    urlAtas.append(temp)
    #print(i)
    
    
    

    # aqui, fazemos a consulta e obtemos o resultado para cada ano em JSON
    

In [ ]:
response = requests.get(url1)
    #print(response.text)

    # Verifica se o tipo de conteúdo é JSON
    
if 'application/json' in response.headers.get('Content-Type', ''):
        if response.content:
            #data = response.json()  # Tenta parsear como JSON
            temp = requests.get(url1, verify=True,headers=chave).json()
            temp=temp[0]['url']
            print(temp)
        else:
            temp=""        
else:
    temp=""
    urlAtas.append(temp)
    #print(i)
    
    
    

    # aqui, fazemos a consulta e obtemos o resultado para cada ano em JSON
    

In [ ]:
df_urlAtas=pd.DataFrame(urlAtas)





In [ ]:
https://pncp.gov.br/api/pncp/v1/orgaos/01612781000138/compras/2022/000021/atas/000001/arquivos

In [ ]:
df=pd.DataFrame(data, df_urlAtas)
df = pd.concat([data, df_urlAtas], axis=1)

df.to_excel("output.xlsx",sheet_name='Sheet_name_1')  

data.head()






## 6.3.8. Consultar Todos Documentos de uma Contratação


In [ ]:
# Vamos definir os termos de nossa consulta

ano=2019 # você pode mudar o ano de acordo com a necessidade
subfuncao=542 #código referente a "controle ambiental"

## 6.4.4. Consultar Atas de Registro de Preço por Compra

In [ ]:
#/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas



numLinhas=DocAta.shape[0]
urlAtasPorCompras=[]
#urlAtasHistoricas=[]
for i in range(numLinhas):
    cnpj=DocAta.loc[i, 'cnpj']
    anoCompra=DocAta.loc[i, 'anoCompra']
    sequencialCompra=DocAta.loc[i, 'sequencialCompra']
    
    url = f'https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas'
    #url_historico = f'https://pncp.gov.br/api/pncp//v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/historico'
   #url = f'https://pncp.gov.br/api/pncp/v1/orgaos/00394452000103/compras/2024/011953/atas/000003/arquivos' 
    #https://pncp.gov.br/api/pncp//v1/orgaos/00394452000103/compras/2024/011953/atas/000003/historico
    
    # esta URL foi gerada a partir da consulta por órgão no swagger
    #temp=url1[0]['url']
    response = requests.get(url, verify=True, headers=chave)
    

    # Verifica se o tipo de conteúdo é JSON
    if 'application/json' in response.headers.get('Content-Type', ''):
        data = response.json()  # Tenta parsear como JSON
        temp = requests.get(url, verify=True,headers=chave).json()
        temp=temp[0]['url']
    else:
        temp=""
    urlAtasPorCompras.append(temp)
    urlAtasPorCompras

In [ ]:
data = pd.DataFrame([dataPublicacaoPncp,numeroControlePNCPAta,numeroControlePNCPCompra,anoAta, numeroAtaRegistroPreco,
                      vigenciaInicio,vigenciaFim,usuario,objetoContratacao,codigoUnidadeOrgao,nomeOrgao,nomeUnidadeOrgao,urlAtas]) # Each list would be added as a row
data = data.transpose() 
data.columns = ['dataPublicacaoPncp','numeroControlePNCPAta','numeroControlePNCPCompra','anoAta', 'numeroAtaRegistroPreco',
                      'vigenciaInicio','vigenciaFim','usuario','objetoContratacao','codigoUnidadeOrgao','nomeOrgao','nomeUnidadeOrgao','urlAtas']
 # Rename the


data.to_excel("output.xlsx",sheet_name='Sheet_name_1')  


# Buscas Específicas

In [ ]:
#--------------------------------------
# Buscando ATAS relativos ao objeto "limpeza"
data=data[data.objetoContratacao.str.contains("LIMPEZA", case=False)].reset_index(drop=True)
cols = [0,1] # numeroControlePNCPAta, anoAta
df = data[data.columns[cols]]
df


In [ ]:
url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701'
page = 429

for i in (N, -1, -1)
url=url+f'&pagina={page}'
#url=f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701&pagina=1'
registro = requests.get(url,verify=True,headers=chave).json()

dados=registro.get('data')

registro


#-----------------------------------
dataPublicacaoPncp=list(map(lambda x:x["dataPublicacaoPncp"],r_orgao))
numeroControlePNCPAta=list(map(lambda x:x["numeroControlePNCPAta"],r_orgao))
numeroControlePNCPCompra=list(map(lambda x:x["numeroControlePNCPCompra"],r_orgao))
numeroAtaRegistroPreco=list(map(lambda x:x["numeroAtaRegistroPreco"],r_orgao))
anoAta=list(map(lambda x:x["anoAta"],r_orgao))
#cancelado=list(map(lambda x:x["cancelado"],r_orgao))
vigenciaInicio=list(map(lambda x:x["vigenciaInicio"],r_orgao))
vigenciaFim=list(map(lambda x:x["vigenciaFim"],r_orgao))
usuario=list(map(lambda x:x["usuario"],r_orgao))
objetoContratacao=list(map(lambda x:x["objetoContratacao"],r_orgao))
nomeOrgao=list(map(lambda x:x["nomeOrgao"],r_orgao))
codigoUnidadeOrgao=list(map(lambda x:x["codigoUnidadeOrgao"],r_orgao))
nomeUnidadeOrgao=list(map(lambda x:x["nomeUnidadeOrgao"],r_orgao))

data = pd.DataFrame([dataPublicacaoPncp,numeroControlePNCPAta,numeroControlePNCPCompra,numeroAtaRegistroPreco,
                     anoAta,vigenciaInicio,vigenciaFim,usuario,objetoContratacao,
                      codigoUnidadeOrgao,nomeOrgao,nomeUnidadeOrgao]) # Each list would be added as a row
data = data.transpose() 
data.columns = ['dataPublicacaoPncp','numeroControlePNCPAta','numeroControlePNCPCompra','numeroAtaRegistroPreco',
                'anoAta','vigenciaInicio','vigenciaFim','usuario','objetoContratacao','codigoUnidadeOrgao',
                'nomeOrgao','nomeUnidadeOrgao']

""" data=data.query("usuario == 'Contratos.gov.br'")
data=data.reset_index(drop=True)
 """
 #data

# Organizando a coluna dataPublicacaoPncp em ordem decrescente
data.sort_values(['dataPublicacaoPncp'], ascending=[False])



#--------------------------------------
# Buscando ATAS relativos ao objeto "limpeza"
data=data[data.objetoContratacao.str.contains("LIMPEZA", case=False)].reset_index(drop=True)
cols = [0,1] # numeroControlePNCPAta, anoAta
df = data[data.columns[cols]]
df

#---------------------------------
# URL BASE= https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra/atas/{sequencialAta}/arquivos
# 01612781000138-1-000020/2022-000001
cnpj=df.numeroControlePNCPAta.str[:14]
sequencialCompra=df.numeroControlePNCPAta.str[17:23]
anoCompra=df.numeroControlePNCPAta.str[24:28]
sequencialAta=df.numeroControlePNCPAta.str[29:35]
DocAta=pd.DataFrame([cnpj,anoCompra, sequencialCompra,sequencialAta])
DocAta = DocAta.transpose() 
DocAta.columns = ['cnpj','anoCompra','sequencialCompra','sequencialAta']
DocAta

#-------------------numLinhas=DocAta.shape[0]
numLinhas=DocAta.shape[0]
urlAtas=[]
#urlAtasHistoricas=[]
for i in range(numLinhas):
    cnpj=DocAta.loc[i, 'cnpj']
    anoCompra=DocAta.loc[i, 'anoCompra']
    sequencialCompra=DocAta.loc[i, 'sequencialCompra']
    sequencialAta=DocAta.loc[i, 'sequencialAta']

    url1 = f'https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/arquivos'
    #url_historico = f'https://pncp.gov.br/api/pncp//v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/historico'
   #url = f'https://pncp.gov.br/api/pncp/v1/orgaos/00394452000103/compras/2024/011953/atas/000003/arquivos' 
    #https://pncp.gov.br/api/pncp//v1/orgaos/00394452000103/compras/2024/011953/atas/000003/historico
    
    # esta URL foi gerada a partir da consulta por órgão no swagger
    #temp=url1[0]['url']
    response = requests.get(url1, verify=True, headers=chave)
    #response2 = requests.get(url_historico, verify=True, headers=chave)
    
    #response = requests.get(url1)

    #print(response.text)

    # Verifica se o tipo de conteúdo é JSON
    
    if 'application/json' in response.headers.get('Content-Type', ''):
        if response.content:
            #data = response.json()  # Tenta parsear como JSON
            temp = response.json()
            temp=temp[0]['url']
            #print(temp)
        else:
            temp=""        
    else:
        temp=""
    urlAtas.append(temp)
    #print(i)
    
    
    


In [ ]:
url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701'
page = 429
url=url+f'&pagina={page}'
#url=f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701&pagina=1'
registro = requests.get(url,verify=True,headers=chave).json()

dados=registro.get('data')

registro


#-----------------------------------
dataPublicacaoPncp=list(map(lambda x:x["dataPublicacaoPncp"],r_orgao))
numeroControlePNCPAta=list(map(lambda x:x["numeroControlePNCPAta"],r_orgao))
numeroControlePNCPCompra=list(map(lambda x:x["numeroControlePNCPCompra"],r_orgao))
numeroAtaRegistroPreco=list(map(lambda x:x["numeroAtaRegistroPreco"],r_orgao))
anoAta=list(map(lambda x:x["anoAta"],r_orgao))
#cancelado=list(map(lambda x:x["cancelado"],r_orgao))
vigenciaInicio=list(map(lambda x:x["vigenciaInicio"],r_orgao))
vigenciaFim=list(map(lambda x:x["vigenciaFim"],r_orgao))
usuario=list(map(lambda x:x["usuario"],r_orgao))
objetoContratacao=list(map(lambda x:x["objetoContratacao"],r_orgao))
nomeOrgao=list(map(lambda x:x["nomeOrgao"],r_orgao))
codigoUnidadeOrgao=list(map(lambda x:x["codigoUnidadeOrgao"],r_orgao))
nomeUnidadeOrgao=list(map(lambda x:x["nomeUnidadeOrgao"],r_orgao))

data = pd.DataFrame([dataPublicacaoPncp,numeroControlePNCPAta,numeroControlePNCPCompra,numeroAtaRegistroPreco,
                     anoAta,vigenciaInicio,vigenciaFim,usuario,objetoContratacao,
                      codigoUnidadeOrgao,nomeOrgao,nomeUnidadeOrgao]) # Each list would be added as a row
data = data.transpose() 
data.columns = ['dataPublicacaoPncp','numeroControlePNCPAta','numeroControlePNCPCompra','numeroAtaRegistroPreco',
                'anoAta','vigenciaInicio','vigenciaFim','usuario','objetoContratacao','codigoUnidadeOrgao',
                'nomeOrgao','nomeUnidadeOrgao']

""" data=data.query("usuario == 'Contratos.gov.br'")
data=data.reset_index(drop=True)
 """
 #data

# Organizando a coluna dataPublicacaoPncp em ordem decrescente
data.sort_values(['dataPublicacaoPncp'], ascending=[False])



#--------------------------------------
# Buscando ATAS relativos ao objeto "limpeza"
data=data[data.objetoContratacao.str.contains("LIMPEZA", case=False)].reset_index(drop=True)
cols = [0,1] # numeroControlePNCPAta, anoAta
df = data[data.columns[cols]]
df

#---------------------------------
# URL BASE= https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra/atas/{sequencialAta}/arquivos
# 01612781000138-1-000020/2022-000001
cnpj=df.numeroControlePNCPAta.str[:14]
sequencialCompra=df.numeroControlePNCPAta.str[17:23]
anoCompra=df.numeroControlePNCPAta.str[24:28]
sequencialAta=df.numeroControlePNCPAta.str[29:35]
DocAta=pd.DataFrame([cnpj,anoCompra, sequencialCompra,sequencialAta])
DocAta = DocAta.transpose() 
DocAta.columns = ['cnpj','anoCompra','sequencialCompra','sequencialAta']
DocAta

#-------------------numLinhas=DocAta.shape[0]
numLinhas=DocAta.shape[0]
urlAtas=[]
#urlAtasHistoricas=[]
for i in range(numLinhas):
    cnpj=DocAta.loc[i, 'cnpj']
    anoCompra=DocAta.loc[i, 'anoCompra']
    sequencialCompra=DocAta.loc[i, 'sequencialCompra']
    sequencialAta=DocAta.loc[i, 'sequencialAta']

    url1 = f'https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/arquivos'
    #url_historico = f'https://pncp.gov.br/api/pncp//v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/historico'
   #url = f'https://pncp.gov.br/api/pncp/v1/orgaos/00394452000103/compras/2024/011953/atas/000003/arquivos' 
    #https://pncp.gov.br/api/pncp//v1/orgaos/00394452000103/compras/2024/011953/atas/000003/historico
    
    # esta URL foi gerada a partir da consulta por órgão no swagger
    #temp=url1[0]['url']
    response = requests.get(url1, verify=True, headers=chave)
    #response2 = requests.get(url_historico, verify=True, headers=chave)
    
    #response = requests.get(url1)

    #print(response.text)

    # Verifica se o tipo de conteúdo é JSON
    
    if 'application/json' in response.headers.get('Content-Type', ''):
        if response.content:
            #data = response.json()  # Tenta parsear como JSON
            temp = response.json()
            temp=temp[0]['url']
            #print(temp)
        else:
            temp=""        
    else:
        temp=""
    urlAtas.append(temp)
    #print(i)
    
    
    


In [26]:


url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701&pagina=1'
registro = requests.get(url,verify=True,headers=chave).json()


totalPaginas=registro['totalPaginas']
print(totalPaginas)

url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701'



434


In [21]:
list(range(totalPaginas, -1, -1) )

[433,
 432,
 431,
 430,
 429,
 428,
 427,
 426,
 425,
 424,
 423,
 422,
 421,
 420,
 419,
 418,
 417,
 416,
 415,
 414,
 413,
 412,
 411,
 410,
 409,
 408,
 407,
 406,
 405,
 404,
 403,
 402,
 401,
 400,
 399,
 398,
 397,
 396,
 395,
 394,
 393,
 392,
 391,
 390,
 389,
 388,
 387,
 386,
 385,
 384,
 383,
 382,
 381,
 380,
 379,
 378,
 377,
 376,
 375,
 374,
 373,
 372,
 371,
 370,
 369,
 368,
 367,
 366,
 365,
 364,
 363,
 362,
 361,
 360,
 359,
 358,
 357,
 356,
 355,
 354,
 353,
 352,
 351,
 350,
 349,
 348,
 347,
 346,
 345,
 344,
 343,
 342,
 341,
 340,
 339,
 338,
 337,
 336,
 335,
 334,
 333,
 332,
 331,
 330,
 329,
 328,
 327,
 326,
 325,
 324,
 323,
 322,
 321,
 320,
 319,
 318,
 317,
 316,
 315,
 314,
 313,
 312,
 311,
 310,
 309,
 308,
 307,
 306,
 305,
 304,
 303,
 302,
 301,
 300,
 299,
 298,
 297,
 296,
 295,
 294,
 293,
 292,
 291,
 290,
 289,
 288,
 287,
 286,
 285,
 284,
 283,
 282,
 281,
 280,
 279,
 278,
 277,
 276,
 275,
 274,
 273,
 272,
 271,
 270,
 269,
 268,
 267

In [4]:
url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701'
page=433
url=url+f'&pagina={page}'
registro = requests.get(url,verify=True,headers=chave).json()
r_orgao=registro.get('data')
r_orgao
dataPublicacaoPncp=list(map(lambda x:x["dataPublicacaoPncp"],r_orgao))


In [5]:
r_orgao

[{'numeroControlePNCPAta': '15126437000143-1-001632/2024-000009',
  'numeroAtaRegistroPreco': '00395',
  'anoAta': 2024,
  'numeroControlePNCPCompra': '15126437000143-1-001632/2024',
  'cancelado': False,
  'dataCancelamento': None,
  'dataAssinatura': '2024-09-23',
  'vigenciaInicio': '2024-09-24',
  'vigenciaFim': '2025-09-24',
  'dataPublicacaoPncp': '2024-09-23',
  'dataInclusao': '2024-09-23',
  'dataAtualizacao': '2024-09-23',
  'usuario': 'Contratos.gov.br',
  'objetoContratacao': 'Aquisição centralizada de FIOS CIRÚRGICOS, por meio de Sistema de Registro de Preços - SRP, com o objetivo de atender prioritariamente o Complexo Hospitalar da UFRJ, com possibilidade de assistir aos demais Hospitais Universitários Federais - HUFs integrantes da Rede da Empresa Brasileira de Serviços Hospitalares - Ebserh.',
  'cnpjOrgao': '15126437000143',
  'nomeOrgao': 'EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES',
  'cnpjOrgaoSubrogado': None,
  'nomeOrgaoSubrogado': None,
  'codigoUnidadeOrgao': 

In [6]:
url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701'

df_compilado = pd.DataFrame()
totalPaginas=433    
for page in range(totalPaginas, -1, -1) : 
    print(page)

    url=url+f'&pagina={page}'
    print(url)
    #url=f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701&pagina=1'
    registro = requests.get(url,verify=True,headers=chave).json()

    r_orgao=registro.get('data')

    #-----------------------------------
    # Verifique se r_orgao não é None antes de fazer a iteração
    if r_orgao:
        dataPublicacaoPncp=list(map(lambda x:x["dataPublicacaoPncp"],r_orgao))
        numeroControlePNCPAta=list(map(lambda x:x["numeroControlePNCPAta"],r_orgao))
        numeroControlePNCPCompra=list(map(lambda x:x["numeroControlePNCPCompra"],r_orgao))
        numeroAtaRegistroPreco=list(map(lambda x:x["numeroAtaRegistroPreco"],r_orgao))
        anoAta=list(map(lambda x:x["anoAta"],r_orgao))
        #cancelado=list(map(lambda x:x["cancelado"],r_orgao))
        vigenciaInicio=list(map(lambda x:x["vigenciaInicio"],r_orgao))
        vigenciaFim=list(map(lambda x:x["vigenciaFim"],r_orgao))
        usuario=list(map(lambda x:x["usuario"],r_orgao))
        objetoContratacao=list(map(lambda x:x["objetoContratacao"],r_orgao))
        nomeOrgao=list(map(lambda x:x["nomeOrgao"],r_orgao))
        codigoUnidadeOrgao=list(map(lambda x:x["codigoUnidadeOrgao"],r_orgao))
        nomeUnidadeOrgao=list(map(lambda x:x["nomeUnidadeOrgao"],r_orgao))

    else:
        # Caso r_orgao seja None, você pode definir variáveis vazias ou outro comportamento apropriado
        dataPublicacaoPncp = []
        numeroControlePNCPAta = []
        numeroControlePNCPCompra = []
        numeroAtaRegistroPreco= []
        anoAta= []
        vigenciaInicio= []
        vigenciaFim= []
        usuario= []
        objetoContratacao= []
        codigoUnidadeOrgao= []
        nomeOrgao= []
        nomeUnidadeOrgao= []
    
    data = pd.DataFrame([dataPublicacaoPncp,numeroControlePNCPAta,numeroControlePNCPCompra,
                        numeroAtaRegistroPreco, anoAta,vigenciaInicio,vigenciaFim,usuario,
                        objetoContratacao,codigoUnidadeOrgao,nomeOrgao,nomeUnidadeOrgao]) # Each list would be added as a row
    data = data.transpose() 

    data.columns = ['dataPublicacaoPncp','numeroControlePNCPAta','numeroControlePNCPCompra',
                    'numeroAtaRegistroPreco','anoAta','vigenciaInicio','vigenciaFim','usuario',
                    'objetoContratacao','codigoUnidadeOrgao','nomeOrgao','nomeUnidadeOrgao']

    print("Dataframe"+str(page)+ "Criado!")

    """ data=data.query("usuario == 'Contratos.gov.br'")
    data=data.reset_index(drop=True)
    """
    #data

    # Organizando a coluna dataPublicacaoPncp em ordem decrescente
    #data.sort_values(['dataPublicacaoPncp'], ascending=[False])



    #--------------------------------------
    # Buscando ATAS relativos ao objeto "limpeza"
    
    
    cols = [1,4] # numeroControlePNCPAta, anoAta
    df = data[data.columns[cols]]
    
    #---------------------------------
    # URL BASE= https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra/atas/{sequencialAta}/arquivos
    # 01612781000138-1-000020/2022-000001
    #df['numeroControlePNCPAta'] = df['numeroControlePNCPAta'].astype(str)
    df['numeroControlePNCPAta'] = df['numeroControlePNCPAta'].fillna('')
    cnpj=df.numeroControlePNCPAta.str[:14]
    sequencialCompra=df.numeroControlePNCPAta.str[17:23]
    anoCompra=df.numeroControlePNCPAta.str[24:28]
    sequencialAta=df.numeroControlePNCPAta.str[29:35]
    DocAta=pd.DataFrame([cnpj,anoCompra, sequencialCompra,sequencialAta])
    DocAta = DocAta.transpose() 
    DocAta.columns = ['cnpj','anoCompra','sequencialCompra','sequencialAta']
    DocAta

    #-------------------numLinhas=DocAta.shape[0]
    numLinhas=DocAta.shape[0]
    urlAtas=[]
    #urlAtasHistoricas=[]
    for i in range(numLinhas):
       # print("Url: "+str(i)+"criada!")
        cnpj=DocAta.loc[i, 'cnpj']
        anoCompra=DocAta.loc[i, 'anoCompra']
        sequencialCompra=DocAta.loc[i, 'sequencialCompra']
        sequencialAta=DocAta.loc[i, 'sequencialAta']

        url1 = f'https://pncp.gov.br/api/pncp/v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/atas/{sequencialAta}/arquivos'
        
        response = requests.get(url1, verify=True, headers=chave)
        
        # Verifica se o tipo de conteúdo é JSON
        
        if 'application/json' in response.headers.get('Content-Type', ''):
            if response.content:
                #data = response.json()  # Tenta parsear como JSON
                temp = response.json()
                temp=temp[0]['url']
                #print(temp)
            else:
                temp=""        
        else:
            temp=""
        urlAtas.append(temp)
        #print(i)

    dados = pd.DataFrame([dataPublicacaoPncp,numeroControlePNCPAta,numeroControlePNCPCompra,anoAta, numeroAtaRegistroPreco,
                        vigenciaInicio,vigenciaFim,usuario,objetoContratacao,codigoUnidadeOrgao,nomeOrgao,nomeUnidadeOrgao,urlAtas]) # Each list would be added as a row
    dados = dados.transpose() 
    dados.columns = ['dataPublicacaoPncp','numeroControlePNCPAta','numeroControlePNCPCompra','anoAta', 'numeroAtaRegistroPreco',
                        'vigenciaInicio','vigenciaFim','usuario','objetoContratacao','codigoUnidadeOrgao','nomeOrgao','nomeUnidadeOrgao','urlAtas']
    # Rename the
    
    
    
    df_compilado = pd.concat([df_compilado, dados])
    
    #print("Dataframe-url: "+ str(i) +" criado!" )
    url = f'https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701'
    df_compilado.reset_index(inplace=True, drop=True) 
    #df_compilado=df_compilado[df_compilado.objetoContratacao.str.contains("outsourcing", case=False)].reset_index(drop=True)
    df_compilado=df_compilado[df_compilado.objetoContratacao.str.contains("limpeza", case=False)].reset_index(drop=True)


    # Filtrando atas oriundas do Compras
    df_compilado=df_compilado.query("usuario == 'Contratos.gov.br'")
    
    df_compilado.to_excel("limpeza.xlsx",sheet_name='Sheet_name_1')
    

  
    
        
        
        


433
https://pncp.gov.br/api/consulta/v1/atas?dataInicial=20240801&dataFinal=20250701&pagina=433
Dataframe433Criado!


AttributeError: Can only use .str accessor with string values!

In [26]:
df_compilado.head()
df=pd.concat([df_compilado,df_compilado])
df.shape

(3000, 13)

In [36]:
df_compilado

,dataPublicacaoPncp,numeroControlePNCPAta,numeroControlePNCPCompra,anoAta,numeroAtaRegistroPreco,vigenciaInicio,vigenciaFim,usuario,objetoContratacao,codigoUnidadeOrgao,nomeOrgao,nomeUnidadeOrgao,urlAtas
0,2024-09-23,15126437000143-1-001632/2024-000009,15126437000143-1-001632/2024,2024,00395,2024-09-24,2025-09-24,Contratos.gov.br,"Aquisição centralizada de FIOS CIRÚRGICOS, por...",155007,EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES,EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES,https://pncp.gov.br/pncp-api/v1/orgaos/1512643...
1,2024-09-23,18593103000178-1-000549/2024-000001,18593103000178-1-000549/2024,2024,154,2024-09-13,2025-09-09,W4M AUTOMACAO E CONTROLE LTDA,REFERE-SE À REGISTRO DE PREÇOS PARA FUTURA E E...,02020000,MUNICIPIO DE MONTE CARMELO,SECRETARIA DA FAZENDA,
2,2024-09-23,15126437000143-1-001968/2024-000001,15126437000143-1-001968/2024,2024,00395,2024-09-24,2025-09-24,Contratos.gov.br,Aquisição de soluções parenterais de pequeno e...,155013,EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES,HOSPITAL UNIVERSITÁRIO ONOFRE LOPES,https://pncp.gov.br/pncp-api/v1/orgaos/1512643...
3,2024-09-23,28615557000156-1-000060/2024-000002,28615557000156-1-000060/2024,2024,72,2024-09-23,2025-09-23,Licitanet Licitações Eletrônicas LTDA,[LICITANET] - Aquisição de Gêneros Alimentício...,1508,MUNICIPIO DE ITAOCARA,MUNICÍPIO DE ITAOCARA/RJ,
4,2024-09-23,88370879000104-1-001710/2024-000001,88370879000104-1-001710/2024,2024,56,2024-09-16,2025-09-16,Tecnosweb - Tecnologia de Gestão,MATERIAL DE CONSTRUÇÃO,82,MUNICIPIO DE SAO SEBASTIAO DO CAI,Município de São Sebastião do Cai,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2024-09-23,00394452000103-1-015355/2024-000013,00394452000103-1-015355/2024,2024,00156,2024-09-19,2025-09-19,Contratos.gov.br,"Aquisição de MATERIAL PARA INSTRUÇÃO, MANOBRA ...",160446,COMANDO DO EXERCITO,62 BATALHAO DE INFANTARIA,https://pncp.gov.br/pncp-api/v1/orgaos/0039445...
496,2024-09-23,00394452000103-1-014655/2024-000021,00394452000103-1-014655/2024,2024,00036,2024-09-24,2025-09-24,Contratos.gov.br,"Aquisição de Material de Expediente, visando a...",160470,COMANDO DO EXERCITO,12 GRUPO DE ARTILHARIA DE CAMPANHA,https://pncp.gov.br/pncp-api/v1/orgaos/0039445...
497,2024-09-23,10651417000178-1-000062/2024-000003,10651417000178-1-000062/2024,2024,00186,2024-09-04,2025-09-03,Contratos.gov.br,Aquisição de mobiliário para o ifgoiano.,158124,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNO...","INST.FED.DE EDUC., CIENCIA E TEC. GOIANO",https://pncp.gov.br/pncp-api/v1/orgaos/1065141...
498,2024-09-23,10651417000178-1-000062/2024-000004,10651417000178-1-000062/2024,2024,00187,2024-09-04,2025-09-03,Contratos.gov.br,Aquisição de mobiliário para o ifgoiano.,158124,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNO...","INST.FED.DE EDUC., CIENCIA E TEC. GOIANO",https://pncp.gov.br/pncp-api/v1/orgaos/1065141...


In [20]:
registro

{'data': [{'numeroControlePNCPAta': '01612781000138-1-000021/2022-000001',
   'numeroAtaRegistroPreco': '3',
   'anoAta': 2023,
   'numeroControlePNCPCompra': '01612781000138-1-000021/2022',
   'cancelado': False,
   'dataCancelamento': None,
   'dataAssinatura': '2023-01-12',
   'vigenciaInicio': '2023-01-12',
   'vigenciaFim': '2025-01-12',
   'dataPublicacaoPncp': '2023-01-12',
   'dataInclusao': '2023-01-12',
   'dataAtualizacao': '2024-01-08',
   'usuario': 'Governançabrasil Tecnologia e Gestão em Serviços',
   'objetoContratacao': 'Despesa empenhada para prestacao de servico de horas maquinas para atender programas como abertura e reabertura de acudes  reabertura de estradas  transporte de silagem  cascalho e demais servicos disponibilizados pelas secretarias municipais de obras e servicos publicos e secretaria de agricultura e meio ambiente de Santiago do Sul  conforme Lei Municipal 1013 2021.',
   'cnpjOrgao': '01612781000138',
   'nomeOrgao': 'MUNICIPIO DE SANTIAGO DO SUL',
  